In [ ]:
!pip install --upgrade d2l==1.0.0a0
!pip install torch
%matplotlib inline
import torch
from d2l import torch as d2l

  Using cached d2l-1.0.0a0-py3-none-any.whl.metadata (366 bytes)
  Using cached jupyter-1.1.1-py2.py3-none-any.whl.metadata (2.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.2/388.2 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 6.5 MB/s eta 0:00:00
  Attempting uninstall: jupyter-client
    Found existing installation: jupyter-client 6.1.12
    Uninstalling jupyter-client-6.1.12:
      Successfully unin

In [ ]:
x = torch.arange(-8.0, 8.0, 0.1, requires_grad=True)
y = torch.relu(x)
d2l.plot(x.detach(), y.detach(), 'x', 'relu(x)', figsize=(5, 2.5))

In [ ]:
print(y.detach())
print(y)

**5.2 Implementation**

In [ ]:
from torch import nn
class MLPScratch(d2l.Classifier):
  def __init__(self, num_inputs, num_outputs, num_hiddens, lr, sigma = 0.01):
    super().__init__()
    self.save_hyperparameters()
    self.W1 = nn.Parameter(torch.randn(num_inputs, num_hiddens) * sigma)
    self.b1 = nn.Parameter(torch.zeros(num_hiddens))
    self.W2 = nn.Parameter(torch.randn(num_hiddens, num_outputs) * sigma)
    self.b2 = nn.Parameter(torch.zeros(num_outputs))

In [ ]:
def relu(X):
  zeros = torch.zeros_like(X)
  return torch.max(zeros,X)

In [ ]:
@d2l.add_to_class(MLPScratch)
def forward(self, X):
    X = X.reshape((-1, self.num_inputs))
    H = relu(torch.matmul(X, self.W1) + self.b1)
    return torch.matmul(H, self.W2) + self.b2

In [ ]:
model = MLPScratch(num_inputs=784, num_outputs=10, num_hiddens=256, lr=0.1)
data = d2l.FashionMNIST(batch_size=256)
trainer = d2l.Trainer(max_epochs=10)
trainer.fit(model, data)

In [ ]:
class MLP(d2l.Classifier):
  def __init__(self, num_outputs, num_hiddens, lr):
    super().__init__()
    self.save_hyperparameters()
    self.net = nn.Sequential(nn.Flatten(), nn.LazyLinear(num_hiddens),
                             nn.ReLU(), nn.LazyLinear(num_hiddens), nn.ReLU(), nn.LazyLinear(num_outputs))

In [ ]:
model = MLP(num_outputs=10, num_hiddens=256, lr=0.1)
trainer.fit(model, data)

**5.6**

In [ ]:
%%capture
!pip install --upgrade d2l==1.0.0a0
import torch
from torch import nn
from d2l import torch as d2l

In [ ]:
def dropout_layer(X, dropout):
    assert 0 <= dropout <= 1
    if dropout == 1: return torch.zeros_like(X)
    mask = (torch.rand(X.shape) > dropout).float()
    return mask * X / (1.0 - dropout)

**Kaggle Competition**

In [ ]:
%matplotlib inline
import pandas as pd
import torch
from torch import nn
from d2l import torch as d2l

In [ ]:
class KaggleHouse(d2l.DataModule):
  def __init__(self, batch_size, train = None, val = None):
    super().__init__()
    self.save_hyperparameters()
    self.raw_train = pd.read_csv(d2l.download(
        d2l.DATA_URL + 'kaggle_house_pred_train.csv', self.root,
        sha1_hash='585e9cc93e70b39160e7921475f9bcd7d31219ce'))
    self.raw_val = pd.read_csv(d2l.download(
        d2l.DATA_URL + 'kaggle_house_pred_test.csv', self.root,
        sha1_hash='fa19780a7b011d9b009e8bff8e99922a8ee2eb90'))

In [ ]:
data = KaggleHouse(batch_size=64)
print(data.raw_train.shape)
print(data.raw_val.shape)

(1460, 81)
(1459, 80)


In [ ]:
print(data.raw_train.iloc[:4, [0,1,2,3,-3,-2,-1]])
print(data.raw_val.iloc[:4, [0,1,2,3,-3,-2,-1]])

   Id  MSSubClass MSZoning  LotFrontage SaleType SaleCondition  SalePrice
0   1          60       RL         65.0       WD        Normal     208500
1   2          20       RL         80.0       WD        Normal     181500
2   3          60       RL         68.0       WD        Normal     223500
3   4          70       RL         60.0       WD       Abnorml     140000
     Id  MSSubClass MSZoning  LotFrontage  YrSold SaleType SaleCondition
0  1461          20       RH         80.0    2010       WD        Normal
1  1462          20       RL         81.0    2010       WD        Normal
2  1463          60       RL         74.0    2010       WD        Normal
3  1464          60       RL         78.0    2010       WD        Normal


In [ ]:
#remove id and label columns
#standardize numerical columns
#replace NAN numerical features by 0
#replace discrete features by object

@d2l.add_to_class(KaggleHouse)
def preprocess(self):
  label = 'SalePrice'
  features = pd.concat((
      self.raw_train.drop(columns = ['Id', label]),
      self.raw_val.drop(columns = ['Id'])
      ))
  numerical_features = features.dtypes[features.dtypes != 'object'].index
  features[numerical_features] = features[numerical_features].apply(
      lambda x : (x - x.mean()) / x.std()
  )
  features[numerical_features] = features[numerical_features].fillna(0)
  features = pd.get_dummies(features, dummy_na = True)

  self.train = features[:self.raw_train.shape[0]].copy()
  self.train[label] = self.raw_train[label]
  self.val = features[self.raw_train.shape[0]:].copy()


In [ ]:
data.get_dataloader(train = True)

torch.Size([1460, 330])
torch.Size([1460, 1])


In [ ]:
# load train/val data depending on arg
# convert data to features and logged predict tensor
@d2l.add_to_class(KaggleHouse)
def get_dataloader(self, train):
  label = 'SalePrice'
  data = self.train if train else self.val
  if label not in data: return
  tensor_loader = lambda x : torch.tensor(x.values.astype(float),
                                          dtype = torch.float32)

  tensors = (tensor_loader(data.drop(columns = [label])),
             torch.log(tensor_loader(data[label])).reshape(-1,1))
  return self.get_tensorloader(tensors,train)